## Movie Recommendation System using Boltzmann Machine

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data
import torch.optim as optim

In [2]:
movie = pd.read_csv('data/movies.dat', sep = '::', engine = 'python', header = None, encoding = 'latin-1')
rating = pd.read_csv('data/ratings.dat', sep = '::', engine = 'python', header = None, encoding = 'latin-1')
user = pd.read_csv('data/users.dat', sep = '::', engine = 'python', header = None, encoding = 'latin-1')

In [3]:
print ("movie data shape" + str(movie.shape))
print ("rating data shape" + str(rating.shape))
print ("user data shape" + str(user.shape))

movie data shape(3883, 3)
rating data shape(1000209, 4)
user data shape(6040, 5)


In [4]:
movie.head(5)

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
rating.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
user.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [7]:
data = pd.read_csv('data-K cross validation/u1.base', delimiter='\t')
data = np.array(data, dtype='int')

In [8]:
test = pd.read_csv('data-K cross validation/u1.test', delimiter='\t')
test = np.array(test, dtype='int')

In [9]:
print (data.shape)
print (test.shape)

(79999, 4)
(19999, 4)


In [10]:
data[:10]

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       [        1,         5,         3, 889751712],
       [        1,         7,         4, 875071561],
       [        1,         8,         1, 875072484],
       [        1,         9,         5, 878543541],
       [        1,        11,         2, 875072262],
       [        1,        13,         5, 875071805],
       [        1,        15,         5, 875071608]])

In [11]:
test[:10]

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       [        1,        17,         3, 875073198],
       [        1,        20,         4, 887431883],
       [        1,        23,         4, 875072895],
       [        1,        24,         3, 875071713],
       [        1,        27,         2, 876892946],
       [        1,        31,         3, 875072144],
       [        1,        33,         4, 878542699]])

In [20]:
total_users = int(max(max(data[:,0]), max(test[:,0])))
total_movies = int(max(max(data[:,1]), max(test[:,1])))

In [21]:
print (users)
print (movies)

943
1682


In [22]:
def convert(inputs):
    new_data = []
    for users in range(1, total_users + 1):
        id_movies = inputs[:,1][inputs[:,0] == users]
        id_ratings = inputs[:,2][inputs[:,0] == users]
        ratings = np.zeros(total_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [23]:
train = convert(data)
test = convert(test)

In [26]:
train[0]

[0.0,
 3.0,
 4.0,
 3.0,
 3.0,
 0.0,
 4.0,
 1.0,
 5.0,
 0.0,
 2.0,
 0.0,
 5.0,
 0.0,
 5.0,
 5.0,
 0.0,
 4.0,
 5.0,
 0.0,
 1.0,
 4.0,
 0.0,
 0.0,
 4.0,
 3.0,
 0.0,
 4.0,
 1.0,
 3.0,
 0.0,
 5.0,
 0.0,
 2.0,
 1.0,
 0.0,
 2.0,
 3.0,
 0.0,
 3.0,
 2.0,
 5.0,
 4.0,
 0.0,
 5.0,
 4.0,
 0.0,
 5.0,
 0.0,
 5.0,
 0.0,
 4.0,
 0.0,
 0.0,
 5.0,
 0.0,
 5.0,
 4.0,
 5.0,
 0.0,
 0.0,
 0.0,
 2.0,
 0.0,
 0.0,
 4.0,
 0.0,
 4.0,
 0.0,
 0.0,
 3.0,
 0.0,
 0.0,
 0.0,
 4.0,
 0.0,
 4.0,
 0.0,
 4.0,
 0.0,
 0.0,
 0.0,
 3.0,
 0.0,
 0.0,
 0.0,
 5.0,
 4.0,
 5.0,
 0.0,
 0.0,
 0.0,
 5.0,
 2.0,
 4.0,
 0.0,
 0.0,
 0.0,
 3.0,
 0.0,
 2.0,
 0.0,
 0.0,
 0.0,
 2.0,
 4.0,
 0.0,
 0.0,
 5.0,
 1.0,
 5.0,
 0.0,
 0.0,
 0.0,
 5.0,
 3.0,
 0.0,
 0.0,
 5.0,
 0.0,
 0.0,
 3.0,
 4.0,
 5.0,
 0.0,
 2.0,
 5.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 4.0,
 0.0,
 4.0,
 3.0,
 5.0,
 1.0,
 3.0,
 0.0,
 3.0,
 2.0,
 0.0,
 4.0,
 0.0,
 4.0,
 3.0,
 0.0,
 2.0,
 0.0,
 0.0,
 5.0,
 3.0,
 0.0,
 0.0,
 4.0,
 0.0,
 3.0,
 0.0,
 0.0,
 0.0,
 4.0,
 0.0,
 0.0,
 5.0,
 5.0,
 2.0